In [7]:
from keras.applications.mobilenet import MobileNet
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

base_model = MobileNet()
print(base_model.summary())

TypeError: depthwise_conv2d() got an unexpected keyword argument 'data_format'

In [ ]:
for layer in base_model.layers[]:
    layer.trainable = False
x = base_model.output
predictions = Dense(1, activation='softmax')(x)

model = Model(inputs=base_model.inputs,
              outputs=predictions)
model.compile(optimizer='adam',
              loss='categorical_crossentropy')


train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
        '../data/raw/DATASET/TRAIN',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        '../data/raw/DATASET/TEST',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

checkpoint = ModelCheckpoint("resnet50.h5", monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=False,
                             mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, 
                      mode='auto')


# Train the model 
model.fit_generator(
    train_generator,
    samples_per_epoch = 2056,
    epochs = 50,
    validation_data = validation_generator,
    nb_val_samples = 256,
    callbacks = [checkpoint, early])


